In [1]:
import numpy as np
import torch 
from torch.utils import data
from d2l import torch as d2l

In [3]:

true_w = torch.tensor([2,-3.4])
true_b = torch.tensor(4.2)

features,labels = d2l.synthetic_data(true_w,true_b,1000)

### 读取数据

In [5]:
def load_array(data_array,batch_size,is_train = True):
    # is_train 表示在训练中，需要打乱数据
    dataset = data.TensorDataset(*data_array)
    """
    TensorDataset 类
        功能：torch.utils.data.TensorDataset 是 Dataset 类的一个具体实现，用于处理张量形式的数据。
        它可以将多个张量组合成一个数据集，每个张量的第一维必须相同，表示样本的数量。
    """
    return data.DataLoader(dataset,batch_size,shuffle = is_train)
"""
torch.utils.data.DataLoader 创建一个数据加载器对象。数据加载器将数据集包装起来，并提供迭代数据集的功能，
每次返回一个大小为 batch_size 的小批量。shuffle=is_train 参数控制是否在每个 epoch 之前随机打乱数据。
"""

'\ntorch.utils.data.DataLoader 创建一个数据加载器对象。数据加载器将数据集包装起来，并提供迭代数据集的功能，\n每次返回一个大小为 batch_size 的小批量。shuffle=is_train 参数控制是否在每个 epoch 之前随机打乱数据。\n\n'

In [7]:
batch_size = 10
data_iter = load_array([features,labels],batch_size)

In [12]:
"""
dataLoader 对象本身就是一个可迭代对象，内部有 __iter__ 方法
使用iter() 就是调用了这个方法，显式的返回一个迭代器。
也可以隐式的进行访问如
for x,y in data_iter:  # for 循环会自动调用__iter__ 
    # 每次处理一小批数据

你可以把 DataLoader 想象成一个可以生产迭代器的工厂 每次调用 iter(data_iter) 都会创建一个新的迭代器，用于遍历 DataLoader 中的数据。  如果你直接调用 next(data_iter)，Python 解释器会尝试将 DataLoader 对象本身当作迭代器来使用，
但这是不行的，因为它没有实现 __next__ 方法（虽然它实现了 __iter__ 方法）。
next() 函数需要一个迭代器作为参数。  DataLoader 对象本身虽然是可迭代的，但它不是一个迭代器对象。  你需要先通过 iter(data_iter) 
获取 DataLoader 的迭代器，然后再使用 next() 函数。
"""

next(iter(data_iter))

[tensor([[ 0.6312, -0.6313],
         [ 0.8496, -0.8005],
         [ 0.2374,  0.3075],
         [ 0.8989, -0.5394],
         [-0.7739,  0.2213],
         [ 0.1109,  0.3202],
         [-0.4851,  0.0514],
         [ 1.0042,  1.3578],
         [ 2.4910,  1.5118],
         [ 1.9617,  1.8128]]),
 tensor([[7.5994],
         [8.6350],
         [3.6251],
         [7.8263],
         [1.9030],
         [3.3300],
         [3.0388],
         [1.5939],
         [4.0414],
         [1.9616]])]

### 定义模型

In [13]:
from torch import nn

### torch.nn 库
https://blog.csdn.net/HiWangWenBing/article/details/120614234 介绍
- torch.nn.Liner(in_features,out_features,bias = True) 全连接
   in_features 是输入的x的维度，也就是特征的个数，自己实现的时候，就是列的个数
- torch.nn.functional nn.functional定义了创建神经网络所需要的一些常见的处理函数
   如 nn.functional.sigmod nn.functional.relu()

In [14]:
net = nn.Sequential(nn.Linear(2,1))
# Sequential 是神经网络每一层的容器，这里我们只有一层全连接
# 输入特征大小为2 输出特征大小1

In [17]:
# 初始化模型参数
net[0].weight.data.normal_(0,0.1)
net[0].bias.data.fill_(0)


tensor([0.])

### 定义损失函数

In [19]:
loss = nn.MSELoss()

### 定义优化函数 随机梯度下降

torch.optim 模块包含了多种优化器类
- torch.optim.SGD（随机梯度下降）
- torch.optim.Adam Adam 是一种自适应学习率的优化算法，它结合了 AdaGrad 和 RMSProp 的优点，能够自适应地调整每个参数的学习率。

In [20]:
trainer = torch.optim.SGD(net.parameters(),lr = 0.03)

### 训练

训练过程：
- 向前传播 net(x)
- 计算损失 loss(net(x),y)
- 梯度清零 trainer.zero_grad() 类似之前parama.grad.zero_()
- 反向传播，梯度清零之后才能反向传播，因为是累加的 l.backward()
- 更新参数 trainer.step()
- 

In [22]:
num_epoch = 3

for epoch in range(num_epoch):
    for x,y in data_iter:
        l = loss(net(x),y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features),labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000319
epoch 2, loss 0.000105
epoch 3, loss 0.000106
